In [1]:
from pyspark.sql import Window
from pyspark.sql.functions import lag, col, lit, lead
from sparkstudy.deploy.demo_sessions import DemoSQLSessionFactory
from sparkstudy.libs.tools import create_random_data
import pyspark.sql.functions as F
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
COLUMNS = ["name","age","salary"]
session_factory_normal = DemoSQLSessionFactory(name="normal")
data = create_random_data(10)
spark_session = session_factory_normal.build_session()
df = spark_session.createDataFrame(data,COLUMNS).cache()
df.show()

KeyboardInterrupt: 

本来想要实验一下怎么shift的。结果发现在spark里面，这类操作必须通过window

In [ ]:
w = Window().partitionBy("name").orderBy(col("name"))
df.select("*", lag("age").over(w).alias("new_col")).show()



类似于禁用orderby

In [ ]:
w = Window().partitionBy().orderBy(lit(1))
df.select("*", lag("age",2).over(w).alias("new_col")).show()

In [ ]:
w = Window().partitionBy().orderBy(lit(1))
df.select("*", lead("age",2).over(w).alias("new_col")).show()


想要测试一下其他的一些东西。比如说是max这类。

- 通过window来计算计算一段区间的最大值。rolling的计算
- 通过行号来修改一些首尾的一些行号。这样可以方便
- 从partition来获取count
- pandas查询数据

In [ ]:
w = Window().partitionBy("name").orderBy(col("name")).rowsBetween(Window.currentRow , 1)
res = df.withColumn("test",F.sum("salary").over(w))
res = res.withColumn('rn',F.row_number().over(Window.partitionBy("name").orderBy('salary')))
res.show()

In [ ]:
count = df.groupBy("name").count()
count.show()

In [ ]:
pd_count = count.toPandas()
pd_count.loc[pd_count['name']=="O","count"].values[0]